In [1]:
import time
import csv
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
//*[@id="__next"]/div/div[2]/div[2]/div[1]
//*[@id="__next"]/div/div[2]/div[2]/div[1]/div/a[1]
//*[@id="__next"]/div/div[2]/div[2]/div[1]/div/a[2]
//*[@id="__next"]/div/div[2]/div[2]/div[1]/div/a[3]


//*[@id="__next"]/div/div[2]/div[2]/div[2]/div
//*[@id="__next"]/div/div[2]/div[2]/div[2]/div/a[1]..datetime
//*[@id="__next"]/div/div[2]/div[2]/div[2]/div/a[2]...title
//*[@id="__next"]/div/div[2]/div[2]/div[2]/div/a[3]....body(content)
